In [370]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
import mne
import matplotlib.pyplot as plt
from scipy import stats, signal
from numpy import save, load
import pandas as pd
import sklearn
from sklearn.svm import SVC

### read dataset X and Y

In [481]:
subject_id=1
base_path = "features_notwelch/"
# edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
# files=len(edf_file_names)
X=load('features_notwelch/data_chb{:02d}/features_{}_00.npy'.format(subject_id,subject_id))
y=load('features_notwelch/data_chb{:02d}/targets_{}_00.npy'.format(subject_id,subject_id))
for subject_id in (1,3,6,7,8,10,14):
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
    files=len(edf_file_names)
    start=0
    if subject_id==1:
        start=1
    for fileno in range(start, files//2):
        X=np.concatenate((X, load('features_notwelch/data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id, fileno))))
        y=np.concatenate((y, load('features_notwelch/data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id, fileno))))
    print(files)
    print(X.shape)


84
(14577, 208)
76
(28145, 208)
36
(52073, 208)
38
(76155, 208)
40
(83290, 208)
50
(101197, 208)
52
(110462, 208)


In [482]:
subject_id=1
base_path = "sum_only/"
# edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
# files=len(edf_file_names)
X_sum=load('sum_only/data_chb{:02d}/features_{}_00.npy'.format(subject_id,subject_id))
y_sum=load('sum_only/data_chb{:02d}/targets_{}_00.npy'.format(subject_id,subject_id))
for subject_id in (1,3,6,7,8,10,14):
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
    files=len(edf_file_names)
    start=0
    if subject_id==1:
        start=1
    for fileno in range(start, files//2):
        X_sum=np.concatenate((X_sum, load('sum_only/data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id, fileno))))
        y_sum=np.concatenate((y_sum, load('sum_only/data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id, fileno))))
    print(files)
    print(X_sum.shape)


84
(14577, 3)
76
(28145, 3)
36
(52073, 3)
38
(76155, 3)
40
(83290, 3)
50
(101197, 3)
52
(110462, 3)


In [483]:
X_shape, y_shape = X.shape, y.shape
print(X_shape, y_shape)

(110462, 208) (110462,)


In [484]:
# def generate_columns():
#     channel_order= {'FP1-F7':0, 'F7-T7':1, 'T7-P7':2, 'P7-O1':3, 'FP1-F3':4, 'F3-C3':5, 'C3-P3':6, 'P3-O1':7, 'FP2-F4':8, 'F4-C4':9, 'C4-P4':10, 'P4-O2':11, 'FP2-F8':12, 'F8-T8':13, 'T8-P8-0':14, 'P8-O2':15, 'FZ-CZ':16, 'CZ-PZ':17, 'P7-T7':18, 'T7-FT9':19, 'FT9-FT10':20, 'FT10-T8':21 ,'T8-P8-1':22 }
#     col_names=[]
#     for ch in channel_order:
#         # print(ch)
#         for i in range(0,9):
#             col_names.append(str(ch)+'-'+str(i))
#     col_names.append('patient')
#     return col_names

def generate_column_names():
    channel_order= {'FP1-F7':0, 'F7-T7':1, 'T7-P7':2, 'P7-O1':3, 'FP1-F3':4, 'F3-C3':5, 'C3-P3':6, 'P3-O1':7, 'FP2-F4':8, 'F4-C4':9, 'C4-P4':10, 'P4-O2':11, 'FP2-F8':12, 'F8-T8':13, 'T8-P8-0':14, 'P8-O2':15, 'FZ-CZ':16, 'CZ-PZ':17, 'P7-T7':18, 'T7-FT9':19, 'FT9-FT10':20, 'FT10-T8':21 ,'T8-P8-1':22 }
    col_names=[]
    for ch in channel_order:
        # print(ch)
        for i in range(0,9):
            col_names.append(str(ch)+'-'+str(i))
    # col_names.append('rms')
    # col_names.append('sum')
    col_names.append('patient')
    # print(col_names.shape)
    return col_names


In [485]:
df_sums =pd.DataFrame(data=X_sum , columns=['sum', 'rms', 'patient'])
df_sums

,sum,rms,patient
0,122056.929688,7.405896,1.0
1,119804.921875,7.694022,1.0
2,81689.562500,6.464692,1.0
3,60762.343750,6.407615,1.0
4,48564.589844,6.165314,1.0
...,...,...,...
110457,13132.047852,0.847515,14.0
110458,20350.298828,1.106131,14.0
110459,19528.882812,1.343929,14.0
110460,16770.392578,1.977222,14.0


In [486]:
df=pd.DataFrame(data=X, columns=generate_column_names())
# df['patient']=1
# df.at[14577:27187,'patient',]=2
# df.at[27187:40755, 'patient']=3
# df.at[40755:97783, 'patient']=4
# df.at[97783:, 'patient']=5

# df=df[np.append(df.columns[:50], ['rms', 'sum', 'patient'])]
# df['patient']=df_sums['patient']
df['sum']=df_sums['sum']
df['rms']=df_sums['rms']
df['target']=y
print(df.shape)
df=df.dropna()
# df=df.drop([1,4], axis=1)
print(df.shape)
df.reset_index(drop=True, inplace=True)

(110462, 211)
(110462, 211)


In [487]:
df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,sum,rms,target
0,0.009253,0.205576,0.015386,0.013003,0.353573,0.068100,0.261663,0.048393,13087.628906,0.009094,...,0.013950,0.344800,0.081632,0.294163,0.059692,2853.293701,1.0,122056.929688,7.405896,0
1,0.010184,0.198871,0.017687,0.014204,0.371332,0.078176,0.244087,0.050380,13272.797852,0.009179,...,0.017124,0.360151,0.080480,0.282719,0.057230,2595.387695,1.0,119804.921875,7.694022,0
2,0.013337,0.209899,0.024952,0.019939,0.382135,0.072422,0.219988,0.053930,9027.048828,0.010076,...,0.017145,0.405914,0.107520,0.246479,0.072111,2669.577881,1.0,81689.562500,6.464692,0
3,0.014367,0.214465,0.024949,0.024575,0.367699,0.068725,0.233418,0.046182,6688.242676,0.017899,...,0.014635,0.405534,0.109894,0.233375,0.072846,2570.697998,1.0,60762.343750,6.407615,0
4,0.024251,0.179508,0.064263,0.053937,0.377249,0.069827,0.200093,0.030606,4267.430176,0.043423,...,0.022323,0.309459,0.085600,0.301347,0.069259,1922.287476,1.0,48564.589844,6.165314,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110457,0.153695,0.185443,0.182544,0.207289,0.194108,0.031253,0.119453,0.022562,257.937256,0.082895,...,0.198526,0.238103,0.030995,0.114659,0.020887,345.400299,14.0,13132.047852,0.847515,0
110458,0.127338,0.161348,0.194171,0.223590,0.193580,0.035099,0.135368,0.026710,306.199768,0.056652,...,0.101892,0.359295,0.048362,0.163300,0.039990,934.927307,14.0,20350.298828,1.106131,0
110459,0.212475,0.200538,0.215722,0.289813,0.092155,0.019873,0.107063,0.019422,297.625916,0.133074,...,0.143244,0.333949,0.042226,0.145928,0.035688,993.386719,14.0,19528.882812,1.343929,0
110460,0.227205,0.221372,0.207406,0.303420,0.066524,0.011218,0.124839,0.010568,349.006165,0.205929,...,0.313678,0.157901,0.014685,0.068349,0.009225,466.282196,14.0,16770.392578,1.977222,0


In [488]:
mean_sums={}
df["mean_sum"]=0
for patient in pd.unique(df['patient']):
    # patdf=df[df[207]==patient]
    # df2["mean_sum"][df2['patient']==patient]=np.mean(df2['sum'][(df2['patient']==patient)&(df2['target']==0)])
    mean_sum=np.mean(df['sum'][(df['patient']==patient)&(df['target']==0)])
    # df2.where(df2['patient']!=patient, mean_sums[int(patient)])
    mean_sums[patient]=(mean_sum)
print(mean_sums)

{1.0: 341725.9808489427, 3.0: 183551.2078667481, 6.0: 825746.9135398668, 7.0: 951636.5401811939, 8.0: 185861.8824585347, 10.0: 222518.6098710113, 14.0: 28535.77624228168}


In [489]:
mean_rmss={}
df["mean_rms"]=0
for patient in pd.unique(df['patient']):
    # patdf=df[df[207]==patient]
    # df2["mean_sum"][df2['patient']==patient]=np.mean(df2['sum'][(df2['patient']==patient)&(df2['target']==0)])
    mean_rms=np.mean(df['rms'][(df['patient']==patient)&(df['target']==0)])
    # df2.where(df2['patient']!=patient, mean_sums[int(patient)])
    mean_rmss[patient]=(mean_rms)
print(mean_rmss)

{1.0: 251.5836950947427, 3.0: 12.775175573289024, 6.0: 225.93468819656715, 7.0: 443.69328131889085, 8.0: 12.10269160905675, 10.0: 58.79644375697765, 14.0: 1.648858093822819}


In [490]:
df['sum_ratio']=0
for patient in pd.unique(df['patient']):
    df['mean_sum'].where(df['patient']!=patient, mean_sums[int(patient)], inplace=True)

    # df2['new']=df2['sum']/df2["mean_sum"]
df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,sum,rms,target,mean_sum,mean_rms,sum_ratio
0,0.009253,0.205576,0.015386,0.013003,0.353573,0.068100,0.261663,0.048393,13087.628906,0.009094,...,0.294163,0.059692,2853.293701,1.0,122056.929688,7.405896,0,341725.980849,0,0
1,0.010184,0.198871,0.017687,0.014204,0.371332,0.078176,0.244087,0.050380,13272.797852,0.009179,...,0.282719,0.057230,2595.387695,1.0,119804.921875,7.694022,0,341725.980849,0,0
2,0.013337,0.209899,0.024952,0.019939,0.382135,0.072422,0.219988,0.053930,9027.048828,0.010076,...,0.246479,0.072111,2669.577881,1.0,81689.562500,6.464692,0,341725.980849,0,0
3,0.014367,0.214465,0.024949,0.024575,0.367699,0.068725,0.233418,0.046182,6688.242676,0.017899,...,0.233375,0.072846,2570.697998,1.0,60762.343750,6.407615,0,341725.980849,0,0
4,0.024251,0.179508,0.064263,0.053937,0.377249,0.069827,0.200093,0.030606,4267.430176,0.043423,...,0.301347,0.069259,1922.287476,1.0,48564.589844,6.165314,0,341725.980849,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110457,0.153695,0.185443,0.182544,0.207289,0.194108,0.031253,0.119453,0.022562,257.937256,0.082895,...,0.114659,0.020887,345.400299,14.0,13132.047852,0.847515,0,28535.776242,0,0
110458,0.127338,0.161348,0.194171,0.223590,0.193580,0.035099,0.135368,0.026710,306.199768,0.056652,...,0.163300,0.039990,934.927307,14.0,20350.298828,1.106131,0,28535.776242,0,0
110459,0.212475,0.200538,0.215722,0.289813,0.092155,0.019873,0.107063,0.019422,297.625916,0.133074,...,0.145928,0.035688,993.386719,14.0,19528.882812,1.343929,0,28535.776242,0,0
110460,0.227205,0.221372,0.207406,0.303420,0.066524,0.011218,0.124839,0.010568,349.006165,0.205929,...,0.068349,0.009225,466.282196,14.0,16770.392578,1.977222,0,28535.776242,0,0


In [491]:
df['rms_ratio']=0
for patient in pd.unique(df['patient']):
    df['mean_rms'].where(df['patient']!=patient, mean_rmss[int(patient)], inplace=True)

    # df2['new']=df2['sum']/df2["mean_sum"]
df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-7,T8-P8-1-8,patient,sum,rms,target,mean_sum,mean_rms,sum_ratio,rms_ratio
0,0.009253,0.205576,0.015386,0.013003,0.353573,0.068100,0.261663,0.048393,13087.628906,0.009094,...,0.059692,2853.293701,1.0,122056.929688,7.405896,0,341725.980849,251.583695,0,0
1,0.010184,0.198871,0.017687,0.014204,0.371332,0.078176,0.244087,0.050380,13272.797852,0.009179,...,0.057230,2595.387695,1.0,119804.921875,7.694022,0,341725.980849,251.583695,0,0
2,0.013337,0.209899,0.024952,0.019939,0.382135,0.072422,0.219988,0.053930,9027.048828,0.010076,...,0.072111,2669.577881,1.0,81689.562500,6.464692,0,341725.980849,251.583695,0,0
3,0.014367,0.214465,0.024949,0.024575,0.367699,0.068725,0.233418,0.046182,6688.242676,0.017899,...,0.072846,2570.697998,1.0,60762.343750,6.407615,0,341725.980849,251.583695,0,0
4,0.024251,0.179508,0.064263,0.053937,0.377249,0.069827,0.200093,0.030606,4267.430176,0.043423,...,0.069259,1922.287476,1.0,48564.589844,6.165314,0,341725.980849,251.583695,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110457,0.153695,0.185443,0.182544,0.207289,0.194108,0.031253,0.119453,0.022562,257.937256,0.082895,...,0.020887,345.400299,14.0,13132.047852,0.847515,0,28535.776242,1.648858,0,0
110458,0.127338,0.161348,0.194171,0.223590,0.193580,0.035099,0.135368,0.026710,306.199768,0.056652,...,0.039990,934.927307,14.0,20350.298828,1.106131,0,28535.776242,1.648858,0,0
110459,0.212475,0.200538,0.215722,0.289813,0.092155,0.019873,0.107063,0.019422,297.625916,0.133074,...,0.035688,993.386719,14.0,19528.882812,1.343929,0,28535.776242,1.648858,0,0
110460,0.227205,0.221372,0.207406,0.303420,0.066524,0.011218,0.124839,0.010568,349.006165,0.205929,...,0.009225,466.282196,14.0,16770.392578,1.977222,0,28535.776242,1.648858,0,0


In [492]:
df['rms_ratio']=df['rms']/df['mean_rms']
df['sum_ratio']=df['sum']/df['mean_sum']

In [493]:
df['seizure_number']=0
df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-8,patient,sum,rms,target,mean_sum,mean_rms,sum_ratio,rms_ratio,seizure_number
0,0.009253,0.205576,0.015386,0.013003,0.353573,0.068100,0.261663,0.048393,13087.628906,0.009094,...,2853.293701,1.0,122056.929688,7.405896,0,341725.980849,251.583695,0.357178,0.029437,0
1,0.010184,0.198871,0.017687,0.014204,0.371332,0.078176,0.244087,0.050380,13272.797852,0.009179,...,2595.387695,1.0,119804.921875,7.694022,0,341725.980849,251.583695,0.350588,0.030582,0
2,0.013337,0.209899,0.024952,0.019939,0.382135,0.072422,0.219988,0.053930,9027.048828,0.010076,...,2669.577881,1.0,81689.562500,6.464692,0,341725.980849,251.583695,0.239050,0.025696,0
3,0.014367,0.214465,0.024949,0.024575,0.367699,0.068725,0.233418,0.046182,6688.242676,0.017899,...,2570.697998,1.0,60762.343750,6.407615,0,341725.980849,251.583695,0.177810,0.025469,0
4,0.024251,0.179508,0.064263,0.053937,0.377249,0.069827,0.200093,0.030606,4267.430176,0.043423,...,1922.287476,1.0,48564.589844,6.165314,0,341725.980849,251.583695,0.142116,0.024506,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110457,0.153695,0.185443,0.182544,0.207289,0.194108,0.031253,0.119453,0.022562,257.937256,0.082895,...,345.400299,14.0,13132.047852,0.847515,0,28535.776242,1.648858,0.460196,0.514001,0
110458,0.127338,0.161348,0.194171,0.223590,0.193580,0.035099,0.135368,0.026710,306.199768,0.056652,...,934.927307,14.0,20350.298828,1.106131,0,28535.776242,1.648858,0.713150,0.670847,0
110459,0.212475,0.200538,0.215722,0.289813,0.092155,0.019873,0.107063,0.019422,297.625916,0.133074,...,993.386719,14.0,19528.882812,1.343929,0,28535.776242,1.648858,0.684365,0.815066,0
110460,0.227205,0.221372,0.207406,0.303420,0.066524,0.011218,0.124839,0.010568,349.006165,0.205929,...,466.282196,14.0,16770.392578,1.977222,0,28535.776242,1.648858,0.587697,1.199146,0


In [494]:
seizure_number=1
for i in range(1, len(df)):
    # if df.iloc[i]['target']==2 and df.iloc[i-1]['target']==2:
    #     df.iloc[i]['seizure_number']=df.iloc[i-1]['seizure_number']
    if df.iloc[i]['target']==2:
        df.at[i,'seizure_number']=seizure_number
    if (df.iloc[i]['target']!=2 and df.iloc[i-1]['target']==2 ):
        seizure_number+=1
# df.to_csv(r'df.csv')


In [495]:
test_df=pd.DataFrame()
for patient in pd.unique(df['patient']):
    # patdf=df[df[207]==patient]
    last_seizure=max(df[df['patient']==patient]['seizure_number'])
    test_df = test_df.append(df[(df['patient']==patient) & (df['seizure_number']==last_seizure)], ignore_index=False)
test_df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-8,patient,sum,rms,target,mean_sum,mean_rms,sum_ratio,rms_ratio,seizure_number
9049,0.487906,0.294157,0.062645,0.243455,0.061901,0.003636,0.013767,0.003427,733.779602,0.385181,...,1923.753174,1.0,28861.935547,6.292221,2,341725.980849,251.583695,0.084459,0.025010,7
9050,0.492928,0.287013,0.072026,0.238879,0.060910,0.002932,0.012587,0.004014,711.672302,0.415166,...,2015.595825,1.0,27132.595703,7.463470,2,341725.980849,251.583695,0.079399,0.029666,7
9051,0.406936,0.237590,0.078928,0.220020,0.104659,0.004607,0.020451,0.006975,398.291138,0.365368,...,1690.714233,1.0,20481.373047,6.541937,2,341725.980849,251.583695,0.059935,0.026003,7
9052,0.377607,0.232304,0.089268,0.232302,0.118813,0.004778,0.024522,0.006443,364.721436,0.288228,...,1452.021973,1.0,18504.693359,6.787939,2,341725.980849,251.583695,0.054151,0.026981,7
9053,0.464701,0.204894,0.068319,0.203080,0.098251,0.004644,0.023169,0.005235,448.785583,0.327284,...,1480.842529,1.0,19626.798828,6.252022,2,341725.980849,251.583695,0.057434,0.024851,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108236,0.350583,0.335095,0.148207,0.243422,0.072715,0.004027,0.018495,0.004422,443.893036,0.310951,...,356.422028,14.0,11823.984375,0.752936,2,28535.776242,1.648858,0.414357,0.456641,47
108237,0.257892,0.372992,0.143660,0.229055,0.104612,0.005055,0.022896,0.004901,440.472992,0.233394,...,465.785309,14.0,13798.040039,0.892335,2,28535.776242,1.648858,0.483535,0.541184,47
108238,0.228357,0.377390,0.137621,0.221662,0.114477,0.006110,0.024464,0.004368,510.064880,0.216920,...,505.763641,14.0,18065.376953,1.142276,2,28535.776242,1.648858,0.633078,0.692768,47
108239,0.227919,0.336646,0.152785,0.235380,0.128835,0.008861,0.032484,0.006166,480.779297,0.168591,...,505.745270,14.0,17551.761719,1.102245,2,28535.776242,1.648858,0.615079,0.668490,47


In [496]:
test_df['target'].unique()

array([2], dtype=int64)

In [497]:
df = df[df.index.isin(test_df.index) == False]

In [498]:
# df.drop('seizure_number', axis='columns', inplace=True)
# test_df.drop('seizure_number', axis='columns', inplace=True)

In [499]:
df.shape

(110056, 216)

In [500]:
df.drop(['target', 'seizure_number'], axis=1).columns

Index(['FP1-F7-0', 'FP1-F7-1', 'FP1-F7-2', 'FP1-F7-3', 'FP1-F7-4', 'FP1-F7-5',
       'FP1-F7-6', 'FP1-F7-7', 'FP1-F7-8', 'F7-T7-0',
       ...
       'T8-P8-1-6', 'T8-P8-1-7', 'T8-P8-1-8', 'patient', 'sum', 'rms',
       'mean_sum', 'mean_rms', 'sum_ratio', 'rms_ratio'],
      dtype='object', length=214)

In [501]:
df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-8,patient,sum,rms,target,mean_sum,mean_rms,sum_ratio,rms_ratio,seizure_number
0,0.009253,0.205576,0.015386,0.013003,0.353573,0.068100,0.261663,0.048393,13087.628906,0.009094,...,2853.293701,1.0,122056.929688,7.405896,0,341725.980849,251.583695,0.357178,0.029437,0
1,0.010184,0.198871,0.017687,0.014204,0.371332,0.078176,0.244087,0.050380,13272.797852,0.009179,...,2595.387695,1.0,119804.921875,7.694022,0,341725.980849,251.583695,0.350588,0.030582,0
2,0.013337,0.209899,0.024952,0.019939,0.382135,0.072422,0.219988,0.053930,9027.048828,0.010076,...,2669.577881,1.0,81689.562500,6.464692,0,341725.980849,251.583695,0.239050,0.025696,0
3,0.014367,0.214465,0.024949,0.024575,0.367699,0.068725,0.233418,0.046182,6688.242676,0.017899,...,2570.697998,1.0,60762.343750,6.407615,0,341725.980849,251.583695,0.177810,0.025469,0
4,0.024251,0.179508,0.064263,0.053937,0.377249,0.069827,0.200093,0.030606,4267.430176,0.043423,...,1922.287476,1.0,48564.589844,6.165314,0,341725.980849,251.583695,0.142116,0.024506,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110457,0.153695,0.185443,0.182544,0.207289,0.194108,0.031253,0.119453,0.022562,257.937256,0.082895,...,345.400299,14.0,13132.047852,0.847515,0,28535.776242,1.648858,0.460196,0.514001,0
110458,0.127338,0.161348,0.194171,0.223590,0.193580,0.035099,0.135368,0.026710,306.199768,0.056652,...,934.927307,14.0,20350.298828,1.106131,0,28535.776242,1.648858,0.713150,0.670847,0
110459,0.212475,0.200538,0.215722,0.289813,0.092155,0.019873,0.107063,0.019422,297.625916,0.133074,...,993.386719,14.0,19528.882812,1.343929,0,28535.776242,1.648858,0.684365,0.815066,0
110460,0.227205,0.221372,0.207406,0.303420,0.066524,0.011218,0.124839,0.010568,349.006165,0.205929,...,466.282196,14.0,16770.392578,1.977222,0,28535.776242,1.648858,0.587697,1.199146,0


### train test split

In [502]:
df_interictal=df[df['target']==0]
df_interictal=df_interictal.sample(frac=0.03)
df_preictal=df[df['target']==2]
interictal_shape, preictal_shape = df_interictal.shape, df_preictal.shape
interictal_shape= df_interictal.shape

print(df_interictal.shape,  df_preictal.shape)


(2725, 216) (2222, 216)


In [503]:
# X_interictal=np.array(df_interictal[df.columns[:-2]]).astype('float32')
# y_interictal=np.array(df_interictal['target']).astype('float32')
# X_preictal_train  =np.array(df_preictal[df.columns[:-2]]).astype('float32')
# y_preictal_train  =np.array(df_preictal['target']).astype('float32')
X_interictal=np.array(df_interictal[df.drop(['target', 'seizure_number'], axis=1).columns]).astype('float32')
y_interictal=np.array(df_interictal['target']).astype('float32')
X_preictal_train  =np.array(df_preictal[df.drop(['target', 'seizure_number'], axis=1).columns]).astype('float32')
y_preictal_train  =np.array(df_preictal['target']).astype('float32')

In [504]:
print(X_interictal.shape)
print(y_interictal.shape)
print(X_preictal_train.shape)
print(y_preictal_train.shape)

(2725, 214)
(2725,)
(2222, 214)
(2222,)


### train test split

In [505]:
from sklearn.model_selection import train_test_split


In [506]:
X_interictal_train, X_interictal_test, y_interictal_train, y_interictal_test =train_test_split(X_interictal,y_interictal,test_size=0.1, random_state=42)
# X_preictal_train, X_preictal_test, y_preictal_train, y_preictal_test=train_test_split(X_preictal, y_preictal,test_size=0.1, random_state=42)

In [507]:
interictal_test_df=pd.DataFrame(data=X_interictal_test)
interictal_test_df['target']=y_interictal_test
interictal_test_df
# test_df=test_df.append(interictal_test_df, ignore_index=True)
test_df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-8,patient,sum,rms,target,mean_sum,mean_rms,sum_ratio,rms_ratio,seizure_number
9049,0.487906,0.294157,0.062645,0.243455,0.061901,0.003636,0.013767,0.003427,733.779602,0.385181,...,1923.753174,1.0,28861.935547,6.292221,2,341725.980849,251.583695,0.084459,0.025010,7
9050,0.492928,0.287013,0.072026,0.238879,0.060910,0.002932,0.012587,0.004014,711.672302,0.415166,...,2015.595825,1.0,27132.595703,7.463470,2,341725.980849,251.583695,0.079399,0.029666,7
9051,0.406936,0.237590,0.078928,0.220020,0.104659,0.004607,0.020451,0.006975,398.291138,0.365368,...,1690.714233,1.0,20481.373047,6.541937,2,341725.980849,251.583695,0.059935,0.026003,7
9052,0.377607,0.232304,0.089268,0.232302,0.118813,0.004778,0.024522,0.006443,364.721436,0.288228,...,1452.021973,1.0,18504.693359,6.787939,2,341725.980849,251.583695,0.054151,0.026981,7
9053,0.464701,0.204894,0.068319,0.203080,0.098251,0.004644,0.023169,0.005235,448.785583,0.327284,...,1480.842529,1.0,19626.798828,6.252022,2,341725.980849,251.583695,0.057434,0.024851,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108236,0.350583,0.335095,0.148207,0.243422,0.072715,0.004027,0.018495,0.004422,443.893036,0.310951,...,356.422028,14.0,11823.984375,0.752936,2,28535.776242,1.648858,0.414357,0.456641,47
108237,0.257892,0.372992,0.143660,0.229055,0.104612,0.005055,0.022896,0.004901,440.472992,0.233394,...,465.785309,14.0,13798.040039,0.892335,2,28535.776242,1.648858,0.483535,0.541184,47
108238,0.228357,0.377390,0.137621,0.221662,0.114477,0.006110,0.024464,0.004368,510.064880,0.216920,...,505.763641,14.0,18065.376953,1.142276,2,28535.776242,1.648858,0.633078,0.692768,47
108239,0.227919,0.336646,0.152785,0.235380,0.128835,0.008861,0.032484,0.006166,480.779297,0.168591,...,505.745270,14.0,17551.761719,1.102245,2,28535.776242,1.648858,0.615079,0.668490,47


In [508]:
X_train = np.concatenate((X_interictal_train, X_preictal_train))
y_train = np.concatenate((y_interictal_train, y_preictal_train))
X_test = np.array(test_df[test_df.drop(['target', 'seizure_number'], axis=1).columns]).astype('float32')
y_test  =np.array(test_df['target']).astype('float32')
X_test = np.concatenate((X_test, X_interictal_test))
y_test  =np.concatenate((y_test, y_interictal_test))

In [509]:
X_test

array([[4.8790631e-01, 2.9415703e-01, 6.2645197e-02, ..., 2.5158369e+02,
        8.4459297e-02, 2.5010446e-02],
       [4.9292761e-01, 2.8701285e-01, 7.2025925e-02, ..., 2.5158369e+02,
        7.9398692e-02, 2.9665951e-02],
       [4.0693572e-01, 2.3758979e-01, 7.8927673e-02, ..., 2.5158369e+02,
        5.9935078e-02, 2.6003024e-02],
       ...,
       [1.8161055e-02, 1.2019242e-01, 8.0024570e-02, ..., 4.4369327e+02,
        7.8268722e-02, 2.4852756e-02],
       [7.4824855e-02, 7.5760946e-02, 5.2212620e-01, ..., 5.8796444e+01,
        5.5209942e-02, 2.6588451e-02],
       [4.3106064e-02, 1.3040492e-01, 6.2795825e-02, ..., 2.2593469e+02,
        2.4539391e-02, 4.3541770e-03]], dtype=float32)

In [510]:
X_train=np.array(X_train).astype('float32')
X_test=np.array( X_test).astype('float32')
y_train=np.array(y_train).astype('float32')
y_test=np.array( y_test).astype('float32')

X_train_shape =X_train.shape
X_test_shape = X_test.shape
y_train_shape =y_train.shape 
y_test_shape = y_test.shape
print(X_train_shape)
print(X_test_shape)
print(y_train_shape)
print(y_test_shape)

(4674, 214)
(679, 214)
(4674,)
(679,)


### Normalization

In [511]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# scaler.transform(X_train)
# scaler.fit(X_test)
# scaler.transform(X_test)

### PCA

In [512]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=100)
# X_train2=pca.fit_transform(X_train)
# X_test2=pca.transform(X_test)

### training the model

In [513]:
from datetime import datetime

In [514]:
cost: 150.0
weight_dict: {0: 1, 2: 10}

In [534]:
before = datetime.now()
before_time =before.strftime("%H:%M:%S")
print(before_time)

clf = SVC(C=2000,kernel='rbf', random_state=42)
clf.fit(X_train, y_train)
trainAcc = clf.score(X_train, y_train)
testAcc = clf.score(X_test, y_test)
print("**SVM Results:**")
print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")

after = datetime.now()
after_time =after.strftime("%H:%M:%S")
print(after_time)

18:54:54
**SVM Results:**
Training Accuracy: 70%
Testing Accuracy: 66%
18:55:05


In [535]:
# import joblib

    # estimator = joblib.load("/my_models/%s.pkl"%dataset_name)
    # print "using trained model"

# print ("saving new model")
# joblib.dump(clf,"my_models/temporal2000Cundersample75f1.pkl")

['my_models/temporal2000Cundersample75f1.pkl']

In [536]:
# loaded_clf = joblib.load("my_models/SVM_blanced_chb04.pkl")
# y_pred=loaded_clf.predict(X_test)
y_pred=clf.predict(X_test)

In [537]:
clf.get_params(deep=True)

{'C': 2000,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 42,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [538]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, y_pred).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))

TN:139, FP:134, FN:96, TP:310


In [539]:
specificity=(tn)/(tn+fp)
sensitivity=(tp)/(tp+fn)
f1_score=sklearn.metrics.f1_score(y_test, y_pred, pos_label=2)

print('specificity= {} , sensitivity= {}, f1= {}'.format(specificity, sensitivity, f1_score))

specificity= 0.5091575091575091 , sensitivity= 0.7635467980295566, f1= 0.7294117647058822


In [540]:
print('C: {}, W: {}'.format(clf.get_params(deep=True)['C'], clf.get_params(deep=True)['class_weight']))
print('X size ={}, y size = {}'.format(X_shape, y_shape))
print('interictal size ={}, preictal size = {}'.format(interictal_shape, preictal_shape))
print('train size ={}, test size = {}'.format(X_train_shape, X_test_shape))

print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))
print('specificity= {} , sensitivity= {}, f1= {}'.format(specificity, sensitivity, f1_score))

C: 2000, W: None
X size =(110462, 208), y size = (110462,)
interictal size =(2725, 216), preictal size = (2222, 216)
train size =(4674, 214), test size = (679, 214)
Training Accuracy: 70%
Testing Accuracy: 66%
TN:139, FP:134, FN:96, TP:310
specificity= 0.5091575091575091 , sensitivity= 0.7635467980295566, f1= 0.7294117647058822


In [541]:
from sklearn.ensemble import RandomForestClassifier

rf_start = datetime.now()
print(rf_start)


rf = RandomForestClassifier(n_estimators=50, max_depth=20, random_state=70)
rf.fit(X_train, y_train)
trainAcc=rf.score(X_train, y_train)
testAcc=rf.score(X_test, y_test)
print("**Random Forest Results:**")
print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")

rf_finish = datetime.now()
rf_time =(rf_finish-rf_start)
print(rf_time)

2021-06-06 18:55:06.695133
**Random Forest Results:**
Training Accuracy: 99%
Testing Accuracy: 58%
0:00:02.433489


In [542]:
rf_y_pred=rf.predict(X_test)

In [543]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, rf_y_pred).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))

TN:233, FP:40, FN:239, TP:167


In [544]:
specificity=(tn)/(tn+fp)
sensitivity=(tp)/(tp+fn)
f1_score=sklearn.metrics.f1_score(y_test, rf_y_pred, pos_label=2)

print('specificity= {} , sensitivity= {}, f1= {}'.format(specificity, sensitivity, f1_score))

specificity= 0.8534798534798534 , sensitivity= 0.41133004926108374, f1= 0.5448613376835237


In [545]:
print('X size ={}, y size = {}'.format(X_shape, y_shape))
print('interictal size ={}, preictal size = {}'.format(interictal_shape, preictal_shape))
print('train size ={}, test size = {}'.format(X_train_shape, X_test_shape))

print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))
print('specificity= {} , sensitivity= {}, f1= {}'.format(specificity, sensitivity, f1_score))

X size =(110462, 208), y size = (110462,)
interictal size =(2725, 216), preictal size = (2222, 216)
train size =(4674, 214), test size = (679, 214)
Training Accuracy: 99%
Testing Accuracy: 58%
TN:233, FP:40, FN:239, TP:167
specificity= 0.8534798534798534 , sensitivity= 0.41133004926108374, f1= 0.5448613376835237


In [546]:
# def f_importances(coef, names):
#     imp = coef
#     imp,names = zip(*sorted(zip(imp,names)))
#     plt.barh(range(len(names)), imp, align='center')
#     plt.yticks(range(len(names)), names)
#     plt.show()
# f_importances(clf.coef_, generate_column_names())

In [547]:
# stop

In [548]:
# df10=pd.DataFrame(data=X_10, columns=generate_column_names())
df10=df[df['patient']==11]
df10=df10[(df10['target']==2) | (df10['target']==0) ]
# df10['target']=y_10
# print(df10.shape)
# df10=df10.dropna()
# print(df10.shape)
# df10.reset_index(drop=True, inplace=True)
# df10=df10[(df10['target']==0) | (df10['target']==2)]
# print(df10['target'].max())
# df10=df10[(df10['patient']==10) ]
df10
# X_10_test  =np.array(test_df[df.columns[-1]]).astype('float32')


,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-8,patient,sum,rms,target,mean_sum,mean_rms,sum_ratio,rms_ratio,seizure_number


In [549]:
df10['patient'].unique()

array([], dtype=float64)

In [550]:
df10[df10.drop('target', axis=1).columns]

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-7,T8-P8-1-8,patient,sum,rms,mean_sum,mean_rms,sum_ratio,rms_ratio,seizure_number


In [551]:
X_10_test  =np.array(df10.drop('target', axis=1)).astype('float32')
y_10_test  =np.array(df10['target']).astype('float32')


In [552]:
test_10=clf.predict(X_10_test)
test_10.shape

ValueError: Found array with 0 sample(s) (shape=(0, 215)) while a minimum of 1 is required.

In [344]:
tes_10_score = clf.score(X_10_test, y_10_test)
tes_10_score

0.9496551724137932

In [345]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_10_test, test_10).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))
specificity=(tn)/(tn+fp)
sensitivity=(tp)/(tp+fn)
f1_score=sklearn.metrics.f1_score(y_10_test, test_10, pos_label=2)

print('specificity= {} , sensitivity= {}, f1= {}'.format(specificity, sensitivity, f1_score))

TN:10991, FP:465, FN:119, TP:25
specificity= 0.9594099162011173 , sensitivity= 0.1736111111111111, f1= 0.07886435331230283


In [346]:
df_results16=pd.DataFrame({'real':y_10_test, 'predicted':test_10})
df_results16


,real,predicted
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
11595,2.0,0.0
11596,2.0,0.0
11597,2.0,0.0
11598,2.0,0.0


In [347]:
def smooth(ser):
    window=5
    i=0
    while i < len(ser)-window:
        if ser[i]==ser[i+window]:
            ser[i:i+window]=ser[i]
        i+=1
    return np.array(ser)

In [348]:
import statistics
def smooth_mode(ser):
    window=50
    i=0
    while i < len(ser)-window:
        ser[i:i+window]=statistics.mode(ser[i:i+window])
        i+=window
    return np.array(ser)

In [349]:
smooth_pred=smooth(test_10)
smooth_pred

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [350]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_10_test, smooth_pred).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))
specificity=(tn)/(tn+fp)
sensitivity=(tp)/(tp+fn)
f1_score=sklearn.metrics.f1_score(y_10_test, smooth_pred, pos_label=2)

print('specificity= {} , sensitivity= {}, f1= {}'.format(specificity, sensitivity, f1_score))

TN:11310, FP:146, FN:133, TP:11
specificity= 0.9872555865921788 , sensitivity= 0.0763888888888889, f1= 0.07308970099667775


In [351]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_10_test, smooth_mode(test_10)).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))
specificity=(tn)/(tn+fp)
sensitivity=(tp)/(tp+fn)
f1_score=sklearn.metrics.f1_score(y_10_test, smooth_mode(test_10), pos_label=2)

print('specificity= {} , sensitivity= {}, f1= {}'.format(specificity, sensitivity, f1_score))

TN:11406, FP:50, FN:138, TP:6
specificity= 0.9956354748603352 , sensitivity= 0.041666666666666664, f1= 0.06


In [352]:
stop

NameError: name 'stop' is not defined

### grid search

In [130]:
from numpy import mean
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [146]:
balance = [{0:10,2:1}, {0:1,2:1}, {0:1,2:10}, {0:1,2:15}, {0:1,2:20}]
costs=[1.0,10.0,100.0, 150.0,1000.0]
# balance = [{0:1,2:10}, {0:1,2:15}]
# costs=[1.0]
param_grid = dict(C=costs, class_weight=balance)

In [147]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [148]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
grid = GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1_macro')


In [151]:
grid_result = grid.fit(X_train, y_train)

In [152]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.548936 using {'C': 1000.0, 'class_weight': {0: 1, 2: 1}}
0.464987 (0.000025) with: {'C': 1.0, 'class_weight': {0: 10, 2: 1}}
0.464987 (0.000025) with: {'C': 1.0, 'class_weight': {0: 1, 2: 1}}
0.217273 (0.004215) with: {'C': 1.0, 'class_weight': {0: 1, 2: 10}}
0.202093 (0.005248) with: {'C': 1.0, 'class_weight': {0: 1, 2: 15}}
0.193375 (0.004963) with: {'C': 1.0, 'class_weight': {0: 1, 2: 20}}
0.467381 (0.001471) with: {'C': 10.0, 'class_weight': {0: 10, 2: 1}}
0.473319 (0.003141) with: {'C': 10.0, 'class_weight': {0: 1, 2: 1}}
0.269558 (0.003700) with: {'C': 10.0, 'class_weight': {0: 1, 2: 10}}
0.240071 (0.005545) with: {'C': 10.0, 'class_weight': {0: 1, 2: 15}}
0.219372 (0.005653) with: {'C': 10.0, 'class_weight': {0: 1, 2: 20}}
0.474830 (0.001837) with: {'C': 100.0, 'class_weight': {0: 10, 2: 1}}
0.491107 (0.004591) with: {'C': 100.0, 'class_weight': {0: 1, 2: 1}}
0.326736 (0.007160) with: {'C': 100.0, 'class_weight': {0: 1, 2: 10}}
0.263315 (0.005673) with: {'C': 100.0, 'cla